# SteamSpy Data Cleaning

In this section, we'll clean the data sets generated from the Steam Store API. Towards the end of this section, we aim to drop the unnecessary columns and expand the data set by adding more interesting columns.

## Import Libraries and Inspect Data

In [7]:
# standard library imports
from ast import literal_eval
import math
import itertools
# import time
# import re

# third-party imports
import numpy as np
import pandas as pd

In [3]:
# read in downloaded data
raw_steamspy_data = pd.read_csv('../data/gathering/steamspy_data.csv')

# print out number of rows and columns
print('Rows:', raw_steamspy_data.shape[0])
print('Columns:', raw_steamspy_data.shape[1])

# view first five rows
raw_steamspy_data.head()

Rows: 59159
Columns: 20


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,212609,5405,0,"10,000,000 .. 20,000,000",9302,136,181,43,999,999,0,"English, French, German, Italian, Spanish - Sp...",Action,12484,"{'Action': 5439, 'FPS': 4854, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,6242,978,0,"5,000,000 .. 10,000,000",1400,1,20,1,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,86,"{'Action': 757, 'FPS': 318, 'Multiplayer': 269..."
2,30,Day of Defeat,Valve,Valve,NaN,5548,612,0,"1,000,000 .. 2,000,000",1624,0,20,0,499,499,0,"English, French, German, Italian, Spanish - Spain",Action,127,"{'FPS': 793, 'World War II': 259, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,2133,474,0,"100,000 .. 200,000",2425,0,11,0,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,5,"{'Action': 633, 'FPS': 146, 'Classic': 112, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,16547,839,0,"2,000,000 .. 5,000,000",1282,98,127,98,499,499,0,"English, French, German, Korean",Action,122,"{'FPS': 899, 'Action': 337, 'Classic': 265, 'S..."


## Remove unwanted columns

There are many columns that overlap with columns in cleaned Steam dataset. We'll thus remove these duplicated columns and columns with too many missing values.

In [4]:
raw_steamspy_data.isnull().sum()

appid                  0
name                   8
developer             16
publisher             56
score_rank         59115
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                  0
initialprice           0
discount               0
languages              3
genre                 17
ccu                    0
tags                   0
dtype: int64

In [5]:
raw_steamspy_data['userscore'].value_counts().head()

0      59115
100        4
95         3
69         2
94         2
Name: userscore, dtype: int64

By inspecting the dataset, we find that there are several columns having too many missing values - `score_rank` and `userscore`, so we want to remove these two columns for now. What's more, some columns like `genre` & `developer` are overlapped with what we had in Steam Dataset, so we will also remove them. Additionally, there are two temporary columns - `average_2weeks` and `median_2weeks`- which only show the game performance in a specific period of time. Since we're only interested in the long-term performance of games not at a specific period of time, we will remove these tow unrelevant columns as well.

In [6]:
drop_cols = [
    'score_rank','userscore', # too many missing values
    'genre', 'developer', 'publisher', 'price', 'initialprice', 'discount', # provided by Steam data
    'average_2weeks', 'median_2weeks' # not interested in temporally specific columns
]

In [7]:
dropped_df = raw_steamspy_data.copy()
    
# handle missing values
dropped_df = dropped_df[(dropped_df['name'].notnull()) & (dropped_df['name'] != 'none')]
dropped_df = dropped_df[dropped_df['languages'].notnull()]
    
# remove unwanted columns
dropped_df = dropped_df.drop(drop_cols, axis=1)

dropped_df.head()

,appid,name,positive,negative,owners,average_forever,median_forever,languages,ccu,tags
0,10,Counter-Strike,212609,5405,"10,000,000 .. 20,000,000",9302,181,"English, French, German, Italian, Spanish - Sp...",12484,"{'Action': 5439, 'FPS': 4854, 'Multiplayer': 3..."
1,20,Team Fortress Classic,6242,978,"5,000,000 .. 10,000,000",1400,20,"English, French, German, Italian, Spanish - Sp...",86,"{'Action': 757, 'FPS': 318, 'Multiplayer': 269..."
2,30,Day of Defeat,5548,612,"1,000,000 .. 2,000,000",1624,20,"English, French, German, Italian, Spanish - Spain",127,"{'FPS': 793, 'World War II': 259, 'Multiplayer..."
3,40,Deathmatch Classic,2133,474,"100,000 .. 200,000",2425,11,"English, French, German, Italian, Spanish - Sp...",5,"{'Action': 633, 'FPS': 146, 'Classic': 112, 'M..."
4,50,Half-Life: Opposing Force,16547,839,"2,000,000 .. 5,000,000",1282,127,"English, French, German, Korean",122,"{'FPS': 899, 'Action': 337, 'Classic': 265, 'S..."


## Processing on the `owners` Column

Here, we will change the original format of **" xxx .. xxx "** to a better format of **" xxx-xxx "** for future use.

In [8]:
dropped_df['owners'] = dropped_df['owners'].str.replace(',', '').str.replace(' .. ', '-', regex=False)

dropped_df[['name', 'owners']].head()

,name,owners
0,Counter-Strike,10000000-20000000
1,Team Fortress Classic,5000000-10000000
2,Day of Defeat,1000000-2000000
3,Deathmatch Classic,100000-200000
4,Half-Life: Opposing Force,2000000-5000000


## Processing on the `tags` column

In [9]:
tags = dropped_df['tags']

print(tags[0])
tags.head()

{'Action': 5439, 'FPS': 4854, 'Multiplayer': 3411, 'Shooter': 3364, 'Classic': 2795, 'Team-Based': 1872, 'First-Person': 1713, 'Competitive': 1614, 'Tactical': 1352, "1990's": 1206, 'e-sports': 1197, 'PvP': 889, 'Old School': 780, 'Military': 636, 'Strategy': 619, 'Survival': 307, 'Score Attack': 293, '1980s': 274, 'Assassin': 230, 'Nostalgia': 145}


0    {'Action': 5439, 'FPS': 4854, 'Multiplayer': 3...
1    {'Action': 757, 'FPS': 318, 'Multiplayer': 269...
2    {'FPS': 793, 'World War II': 259, 'Multiplayer...
3    {'Action': 633, 'FPS': 146, 'Classic': 112, 'M...
4    {'FPS': 899, 'Action': 337, 'Classic': 265, 'S...
Name: tags, dtype: object

Each value in tags column contains two information:
* Tags that are relevant to that game
* and Number of votes for each tag

Therefore, we want to first keep the tags with high votes for each game in the original dataset. And then export another dataframe which contain the number of votes for each tag for future use as following:

appid | name | Action | FPS | Multiplayer | ...
--- | --- | --- | --- | --- | ---
10 | Counter-Strike | 5439 | 4854 | 0 | ...
20 | Team Fortress Classic | 757  | 318 | 0 | ...
... | ... | ... | ... | ... | ...

In [10]:
tag_data = dropped_df[['appid', 'name', 'tags']].copy()
        
def parse_export_tags(x):
    """
    expand the values in tags column.
    """
    x = literal_eval(x)

    if isinstance(x, dict):
        return x
    elif isinstance(x, list):
        return {}
    else:
        raise TypeError('Something other than dict or list found')

tag_data['tags'] = tag_data['tags'].apply(parse_export_tags)

cols = set(itertools.chain(*tag_data['tags']))

for col in sorted(cols):
    # standardise column names
    col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

    # store the values of that tag or return 0 if it's not inside
    tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)

tag_data = tag_data.drop('tags', axis=1)

import warnings
warnings.filterwarnings('ignore')

tag_data.head()

/var/folders/qr/lskgbhh917vfsngdwpbm94zc0000gn/T/ipykernel_15282/1720550713.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)
/var/folders/qr/lskgbhh917vfsngdwpbm94zc0000gn/T/ipykernel_15282/1720550713.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)
/var/folders/qr/lskgbhh917vfsngdwpbm94zc0000gn/T/ipykernel_15282/1720550713.py:25: PerformanceWarnin

,appid,name,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,...,well_written,werewolves,western,wholesome,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,Counter-Strike,274,1206,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1197
1,20,Team Fortress Classic,0,147,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,0,0,0,0,0,0,0,...,0,0,0,0,0,14,259,0,0,0
3,40,Deathmatch Classic,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,144,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
tag_df = dropped_df.copy()

def parse_tags(x):
        x = literal_eval(x)
        
        if isinstance(x, dict):
            return ';'.join(list(x.keys())[:3])
        else:
            return np.nan
    
tag_df['tags'] = tag_df['tags'].apply(parse_tags)
    
# rows with null tags seem to be superseded by newer release, so remove (e.g. dead island)
tag_df = tag_df[tag_df['tags'].notnull()]

tag_df.head()

,appid,name,positive,negative,owners,average_forever,median_forever,languages,ccu,tags
0,10,Counter-Strike,212609,5405,10000000-20000000,9302,181,"English, French, German, Italian, Spanish - Sp...",12484,Action;FPS;Multiplayer
1,20,Team Fortress Classic,6242,978,5000000-10000000,1400,20,"English, French, German, Italian, Spanish - Sp...",86,Action;FPS;Multiplayer
2,30,Day of Defeat,5548,612,1000000-2000000,1624,20,"English, French, German, Italian, Spanish - Spain",127,FPS;World War II;Multiplayer
3,40,Deathmatch Classic,2133,474,100000-200000,2425,11,"English, French, German, Italian, Spanish - Sp...",5,Action;FPS;Classic
4,50,Half-Life: Opposing Force,16547,839,2000000-5000000,1282,127,"English, French, German, Korean",122,FPS;Action;Classic


In [13]:
tag_data.to_csv('../data/cleaned/steamspy_tag_data.csv', index=False)
tag_df.to_csv('../data/cleaned/steamspy_data_cleaned.csv', index=False)

## Merge and Export the Final Clean Dataset

Here, we will merge the steam-data and steamspy-data together and rename the columns for future analysis.

In [8]:
steam_data = pd.read_csv('../data/cleaned/steam_data_cleaned.csv')
steamspy_data = pd.read_csv('../data/cleaned/steamspy_data_cleaned.csv')

In [9]:
merged = steam_data.merge(steamspy_data, left_on='steam_appid', right_on='appid', suffixes=('', '_steamspy'))
merged.head(3)

,name,steam_appid,required_age,platforms,categories,genres,achievements,release_date,price,english,...,appid,name_steamspy,positive,negative,owners,average_forever,median_forever,languages,ccu,tags
0,Counter-Strike,10,0.0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,2000-11-01,8.19,1,...,10,Counter-Strike,212609,5405,10000000-20000000,9302,181,"English, French, German, Italian, Spanish - Sp...",12484,Action;FPS;Multiplayer
1,Team Fortress Classic,20,0.0,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1999-04-01,3.99,1,...,20,Team Fortress Classic,6242,978,5000000-10000000,1400,20,"English, French, German, Italian, Spanish - Sp...",86,Action;FPS;Multiplayer
2,Day of Defeat,30,0.0,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,0,2003-05-01,3.99,1,...,30,Day of Defeat,5548,612,1000000-2000000,1624,20,"English, French, German, Italian, Spanish - Spain",127,FPS;World War II;Multiplayer


In [10]:
# remove overlapping columns
steam_clean = merged.drop(['name_steamspy', 'steam_appid'], axis=1)

# reindex to reorder columns
steam_clean = steam_clean[[
    'appid',
    'name',
    'release_date',
    'owners',
    'ccu',
    'price',
    'languages',
    'english',
    'developer',
    'publisher',
    'platforms',
    'required_age',
    'categories',
    'genres',
    'tags',
    'achievements',
    'positive',
    'negative',
    'average_forever',
    'median_forever'
]]

steam_clean = steam_clean.rename({
    'tags': 'steamspy_tags',
    'positive': 'positive_ratings',
    'negative': 'negative_ratings',
    'average_forever': 'average_playtime',
    'median_forever': 'median_playtime'
}, axis=1)

steam_clean.head()

,appid,name,release_date,owners,ccu,price,languages,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime
0,10,Counter-Strike,2000-11-01,10000000-20000000,12484,8.19,"English, French, German, Italian, Spanish - Sp...",1,Valve,Valve,windows;mac;linux,0.0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,212609,5405,9302,181
1,20,Team Fortress Classic,1999-04-01,5000000-10000000,86,3.99,"English, French, German, Italian, Spanish - Sp...",1,Valve,Valve,windows;mac;linux,0.0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,0,6242,978,1400,20
2,30,Day of Defeat,2003-05-01,1000000-2000000,127,3.99,"English, French, German, Italian, Spanish - Spain",1,Valve,Valve,windows;mac;linux,0.0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,5548,612,1624,20
3,40,Deathmatch Classic,2001-06-01,100000-200000,5,3.99,"English, French, German, Italian, Spanish - Sp...",1,Valve,Valve,windows;mac;linux,0.0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Classic,0,2133,474,2425,11
4,50,Half-Life: Opposing Force,1999-11-01,2000000-5000000,122,3.99,"English, French, German, Korean",1,Gearbox Software,Valve,windows;mac;linux,0.0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Classic,0,16547,839,1282,127


## Data Processing on `owners` and `ratings`

* `owners`: we'd like to keep the median number of the owner for further evaluation
* `rating`: we will use the [SteamDB](https://steamdb.info/blog/steamdb-rating/) method to generate the total and average rating for each game for the use of analysing their performance.

In [12]:
def parse_rating(df):
    """
    Get the rating from positive and negative rating columns based on SteamDB method
    """
    pos = int(df['positive_ratings'])
    neg = int(df['negative_ratings'])
    
    total_rating = pos + neg
    if total_rating != 0:
        average = pos / total_rating
    else:
        average = 0
    
    score = average - (average*0.5) * 2 ** (-math.log10(total_rating + 1))
    return score *100


def parse_owners(df):
    """
    Get the median number of owners.
    """
    upper_bound, lower_bound = df['owners'].split('-')
    median = (int(upper_bound) + int(lower_bound)) / 2
    return median

steam_clean['ratings'] = steam_clean.apply(parse_rating, axis=1)
steam_clean['median_owners'] = steam_clean.apply(parse_owners, axis=1)
steam_clean['total_ratings'] = steam_clean['positive_ratings'] + steam_clean['negative_ratings']

In [14]:
steam_clean.head()

,appid,name,release_date,owners,ccu,price,languages,english,developer,publisher,...,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,ratings,median_owners,total_ratings
0,10,Counter-Strike,2000-11-01,10000000-20000000,12484,8.19,"English, French, German, Italian, Spanish - Sp...",1,Valve,Valve,...,Action,Action;FPS;Multiplayer,0,212609,5405,9302,181,96.315702,15000000.0,218014
1,20,Team Fortress Classic,1999-04-01,5000000-10000000,86,3.99,"English, French, German, Italian, Spanish - Sp...",1,Valve,Valve,...,Action,Action;FPS;Multiplayer,0,6242,978,1400,20,83.474384,7500000.0,7220
2,30,Day of Defeat,2003-05-01,1000000-2000000,127,3.99,"English, French, German, Italian, Spanish - Spain",1,Valve,Valve,...,Action,FPS;World War II;Multiplayer,0,5548,612,1624,20,86.808615,1500000.0,6160
3,40,Deathmatch Classic,2001-06-01,100000-200000,5,3.99,"English, French, German, Italian, Spanish - Sp...",1,Valve,Valve,...,Action,Action;FPS;Classic,0,2133,474,2425,11,77.986329,150000.0,2607
4,50,Half-Life: Opposing Force,1999-11-01,2000000-5000000,122,3.99,"English, French, German, Korean",1,Gearbox Software,Valve,...,Action,FPS;Action;Classic,0,16547,839,1282,127,92.656284,3500000.0,17386


In [13]:
# export clean dataset
steam_clean.to_csv('../data/cleaned/steam_clean.csv', index=False)